In [1]:

import pandas as pd


In [2]:


def create_transition_matrix(data, states, state_to_index):
    transition_matrix = np.zeros((len(states), len(states)))

    for i in range(len(data) - 1):
        current_state = (data.iloc[i]['runs'], data.iloc[i]['wkt'])
        next_state = (data.iloc[i+1]['runs'], data.iloc[i+1]['wkt'])
        transition_matrix[state_to_index[current_state], state_to_index[next_state]] += 1

    transition_matrix = transition_matrix / transition_matrix.sum(axis=1, keepdims=True)

    return transition_matrix


In [4]:

def create_top1_matrices(data, states, state_to_index):
    matrices = {}

    for batsman_tag in ['RHB', 'LHB']:
        for bowler_tag in ['LAFM', 'RAFM', 'RAO', 'RAOB', 'LAO', 'LAOB', 'LAM', 'RAM']:
            filtered_data = data[(data['batsman_tag'] == batsman_tag) & (data['bowler_tag'] == bowler_tag)]
            if len(filtered_data) > 0:
                matrices[(batsman_tag, bowler_tag)] = create_transition_matrix(filtered_data, states, state_to_index)

    return matrices


In [ ]:

# Create the Top-1 level transition matrices
top1_matrices = create_top1_matrices(data, states, state_to_index)


In [ ]:

# Example batsman-bowler combinations for the first and second innings
first_innings_batsman_bowler_combinations = [('RHB', 'RAFM')] * num_balls  # Replace this with the actual combinations
second_innings_batsman_bowler_combinations = [('LHB', 'RAO')] * num_balls  # Replace this with the actual combinations

# Simulate the first innings using the top-1 level matrices
simulated_first_innings = simulate_ball_by_ball(
    first_innings_transition_matrix,
    state_to_index,
    states,
    num_balls,
    top1_matrices=top1_matrices,
    batsman_bowler_combinations=first_innings_batsman_bowler_combinations,
)

# Simulate the second innings using the top-1 level matrices
simulated_second_innings = simulate_ball_by_ball(
    second_innings_transition_matrix,
    state_to_index,
    states,
    num_balls,
    target=first_innings_total_runs + 1,
    top1_matrices=top1_matrices,
    batsman_bowler_combinations=second_innings_batsman_bowler_combinations,
)


In [ ]:

def simulate_ball_by_ball(transition_matrix, state_to_index, states, num_balls, target=None, top1_matrices=None, batsman_bowler_combinations=None):
    current_state = (0, 0)
    simulation = []
    runs = 0
    wickets = 0

    # Keep track of the current batsman on strike and non-striker
    current_batsman = 0
    current_non_striker = 1

    for ball in range(num_balls):
        if top1_matrices is not None and batsman_bowler_combinations is not None:
            batsman_tag, bowler_tag = batsman_bowler_combinations[current_batsman]
            combination_matrix = top1_matrices.get((batsman_tag, bowler_tag), None)
            if combination_matrix is not None:
                transition_matrix = combination_matrix

        next_state_probs = transition_matrix[state_to_index[current_state]]
        next_state_index = np.random.choice(range(len(states)), p=next_state_probs)
        next_state = states[next_state_index]
        simulation.append(next_state)
        current_state = next_state
        runs += next_state[0]
        wickets += next_state[1]

        # Update the batsman and non-striker positions based on runs scored
        if next_state[0] in [1, 3]:
            current_batsman, current_non_striker = current_non_striker, current_batsman

        # Account for a new batsman coming in after a wicket
        if next_state[1] == 1:
            current_batsman = wickets + 1

        # Change strike at the end of an over
        if (ball + 1) % 6 == 0:
            current_batsman, current_non_striker = current_non_striker, current_batsman

        if target is not None:
            if runs >= target or wickets == 10:
                break

    return simulation

